In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Installing necessary packages

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 21.9 MB/s 
     |████████████████████████████████| 6.6 MB 66.4 MB/s 
     |████████████████████████████████| 120 kB 68.1 MB/s 


In [ ]:
! pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 27.0 MB/s 


In [ ]:
! pip install sentence-splitter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 2.2 MB/s 


In [ ]:
pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=4d5387e0147ffd93c8a7d154599629cd18c2bab3b8ecf461ae504c832938cd10
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [ ]:
! pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 28.0 MB/s 


In [ ]:
! pip install git+https://github.com/PrithivirajDamodaran/Parrot.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PrithivirajDamodaran/Parrot.git to /tmp/pip-req-build-lm87u3w2
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot.git /tmp/pip-req-build-lm87u3w2
     |████████████████████████████████| 50 kB 5.8 MB/s 
     |████████████████████████████████| 85 kB 5.4 MB/s 
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8602 sha256=861b061937c79f82eac8d38278ce816cd618afca4d4c3a34447f4d2971c1bb47
  Stored in directory: /tmp/pip-ephem-wheel-cache-qjmbw5mt/wheels/d8/27/d8/b757755a5b0b804c59714fdd8a4e2733c20802f638f17f7d96
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149866 sha256=14b61d495e1bad1d54fc6ca104e2cd75c006d5b2d7ab0fc9cc14b1360156ff26
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
  Created wheel for 

# Libraries

In [ ]:
import csv
import pandas as pd
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
from parrot import Parrot
from torchmetrics.text.rouge import ROUGEScore
from torchmetrics.text.bert import BERTScore
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

# Load the data

In [ ]:
data1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/2_Copy of turkers_annotations - turkers_annotations.tsv', sep='\t', error_bad_lines = False)

# Data: [https://github.com/yanaiela/num_fh/blob/master/data/resolution/raw/turkers_annotations.csv] --> some columns are deleted

In [ ]:
data1.head(5)

,Input.task_id,Input.show_id,Input.scene_index,Input.text_index,Input.target_sentence,Input.text,Input.text_reduced,Input.text_html,Input.text_reduced_html,Input.target,Input.target_start_index,Input.target_end_index,Answer.ans
0,0,58,1,1,"Okay, remember that the garbage goes out on Tu...","[first lines] Joy Stark: Okay, remember that t...","[first lines] Joy Stark: Okay, remember that t...","[first lines]<br/>Joy Stark: Okay, remember th...","[first lines]<br/>Joy Stark: Okay, remember th...",911,30.0,31.0,5.0
1,0,58,1,1,"Okay, remember that the garbage goes out on Tu...","[first lines] Joy Stark: Okay, remember that t...","[first lines] Joy Stark: Okay, remember that t...","[first lines]<br/>Joy Stark: Okay, remember th...","[first lines]<br/>Joy Stark: Okay, remember th...",911,30.0,31.0,5.0
2,0,58,1,1,"Okay, remember that the garbage goes out on Tu...","[first lines] Joy Stark: Okay, remember that t...","[first lines] Joy Stark: Okay, remember that t...","[first lines]<br/>Joy Stark: Okay, remember th...","[first lines]<br/>Joy Stark: Okay, remember th...",911,30.0,31.0,5.0
3,1,62,0,6,Because we owe them 1200.,[first lines] Eddie Stark: 'Morning! Joy Stark...,Joy Stark: I'm paying our credit card bill. Ed...,[first lines]<br/>Eddie Stark: 'Morning!<br/>J...,Joy Stark: I'm paying our credit card bill.<br...,1200,4.0,5.0,4.0
4,1,62,0,6,Because we owe them 1200.,[first lines] Eddie Stark: 'Morning! Joy Stark...,Joy Stark: I'm paying our credit card bill. Ed...,[first lines]<br/>Eddie Stark: 'Morning!<br/>J...,Joy Stark: I'm paying our credit card bill.<br...,1200,4.0,5.0,4.0


In [ ]:
data1.shape

(25274, 13)

In [ ]:
data2=  pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/full_pos.tsv', sep='\t', error_bad_lines = False)

# Data: [https://github.com/yanaiela/num_fh/blob/master/data/identification/processed/full_pos.tsv]

b'Skipping line 987: expected 8 fields, saw 10\nSkipping line 2052: expected 8 fields, saw 9\nSkipping line 7541: expected 8 fields, saw 9\nSkipping line 10353: expected 8 fields, saw 9\nSkipping line 11962: expected 8 fields, saw 9\nSkipping line 12492: expected 8 fields, saw 10\nSkipping line 16463: expected 8 fields, saw 9\nSkipping line 18364: expected 8 fields, saw 9\nSkipping line 20148: expected 8 fields, saw 9\nSkipping line 21767: expected 8 fields, saw 9\nSkipping line 21989: expected 8 fields, saw 9\nSkipping line 24845: expected 8 fields, saw 9\nSkipping line 25181: expected 8 fields, saw 9\nSkipping line 26118: expected 8 fields, saw 9\nSkipping line 28845: expected 8 fields, saw 9\nSkipping line 32687: expected 8 fields, saw 9\nSkipping line 35549: expected 8 fields, saw 9\nSkipping line 39105: expected 8 fields, saw 9\nSkipping line 39834: expected 8 fields, saw 9\nSkipping line 40302: expected 8 fields, saw 9\nSkipping line 43608: expected 8 fields, saw 9\nSkipping line

# Data manipulation

In [ ]:
data1 = data1.drop_duplicates(keep='first')

In [ ]:
data1.shape

(11292, 13)

In [ ]:
data1['id'] = data1['Input.show_id'].map(str) + '-' + data1['Input.scene_index'].map(str) + '-' + data1['Input.text_index'].map(str)

In [ ]:
data1= data1[['Input.text', 'id']]

In [ ]:
data2.shape

(79432, 8)

In [ ]:
data2 = data2.drop_duplicates(keep='first')

In [ ]:
data2.shape

(79432, 8)

In [ ]:
data2['id'] = data2['id'].map(str) + '-' + data2['scene_ind'].map(str) + '-' + data2['sentence_ind'].map(str)

In [ ]:
data2 = data2[['text', 'id']]

In [ ]:
data2 = data2.head(300)

In [ ]:
# Combining two datasets:

data = pd.merge(data1, data2, on='id')

In [ ]:
data

,Input.text,id,text
0,"[first lines] Joy Stark: Okay, remember that t...",58-1-1,"Okay, remember that the garbage goes out on Tu..."
1,[first lines] Eddie Stark: 'Morning! Joy Stark...,62-0-6,Because we owe them 1200.
2,FBI Agent Nicole Scott: You want another crool...,75-1-3,I've had two already. Who eats three croolers ...
3,Janey Cooper: Would you like a muffin? Jess Ma...,75-5-4,Try one.
4,Lucky: Red Airedale rules! Cadpig: Yellow's my...,107-5-2,I like the brown one. It looks like gravy.
...,...,...,...
320,"Jack Donaghy: Lemon, let me tell you a little ...",573-7-0,"Lemon, let me tell you a little story. It was ..."
321,"Liz Lemon: Okay, very funny. You bought a page...",576-1-1,"Oh, I can't. I'm expecting a call from 1983."
322,"Cerie: Congratulations, Liz. It's inspiring to...",591-3-1,"I'm 42, Cerie."
323,"Liz Lemon: Look, I know she gets under your sk...",591-6-0,"Look, I know she gets under your skin, but you..."


In [ ]:
data.to_csv('/content/gdrive/My Drive/Colab Notebooks/sentences_with_context.tsv')

# Generate paraphrases using BART, Pegasus, Parrot

In [ ]:
import os
model = BartForConditionalGeneration.from_pretrained('eugenesiow/bart-paraphrase')
#CUDA_LAUNCH_BLOCKING=1
#os.system('CUDA_LAUNCH_BLOCKING=1')
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
#model.to('cuda')
tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')

list_paraphrases = []
for index, row in data[0:300].iterrows():
    batch = tokenizer(row['Input.text'], return_tensors='pt', max_length=1024, truncation=True, padding='max_length')
    #batch = batch.to('cuda')
    generated_ids = model.generate(batch['input_ids'])
    generated_sentence = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    list_paraphrases.append((row['Input.text'], row['id'], row['text'], generated_sentence))

In [ ]:
header = ['Paragraph', 'Id', 'Sentence', 'BART_Paragraph']
with open('/content/gdrive/My Drive/Colab Notebooks/generation_context_2.csv', 'w') as f:
  writer = csv.writer(f)
  writer.writerow(header)
  for i in list_paraphrases:
    writer.writerow(i)


In [ ]:
data2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/generation_context_2.csv')

In [ ]:
data2

,Paragraph,Id,Sentence,BART_Paragraph
0,"[first lines] Joy Stark: Okay, remember that t...",58-1-1,"Okay, remember that the garbage goes out on Tu...","['Joy Stark: Okay, remember that the garbage g..."
1,[first lines] Eddie Stark: 'Morning! Joy Stark...,62-0-6,Because we owe them 1200.,"[""[first lines] Eddie Stark: 'Morning! Joy Sta..."
2,FBI Agent Nicole Scott: You want another crool...,75-1-3,I've had two already. Who eats three croolers ...,['FBI Agent Nicole Scott: You want another cro...
3,Janey Cooper: Would you like a muffin? Jess Ma...,75-5-4,Try one.,['Janey Cooper: Would you like a muffin? Jess ...
4,Lucky: Red Airedale rules! Cadpig: Yellow's my...,107-5-2,I like the brown one. It looks like gravy.,"[""Lucky: Red Airedale rules! Cadpig: Yellow's ..."
...,...,...,...,...
295,Sheriff Neil Prescott: It's against the law to...,495-1-4,A man in trouble generally runs to his friends...,['Sheriff Neil Prescott: What makes you think ...
296,Ranger Clint Travis: The sign outside says you...,500-1-3,You can find an easier one - being a ranger is...,['Ranger Clint Travis: The sign outside says y...
297,Banker: The Papago chiefs distributed the mone...,505-0-1,Or a dry one.,['Banker: The Papago Chiefs distributed the mo...
298,"President George W Bush: General, call off the...",508-8-10,Yeah the speedy one!,"['President George W Bush: General, call off t..."


In [ ]:
#model.to('cuda')
list_paraphrases2 = []
for index, row in data2[0:300].iterrows():
    batch = tokenizer(row['Sentence'], return_tensors='pt')
    #batch = batch.to('cuda')
    generated_ids = model.generate(batch['input_ids'])
    generated_sentence = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    list_paraphrases2.append(generated_sentence)

In [ ]:
data2['BART_Sentence'] = list_paraphrases2

In [ ]:
data2

,Paragraph,Id,Sentence,BART_Paragraph,BART_Sentence
0,"[first lines] Joy Stark: Okay, remember that t...",58-1-1,"Okay, remember that the garbage goes out on Tu...","['Joy Stark: Okay, remember that the garbage g...","[Okay, remember that the garbage goes out on T..."
1,[first lines] Eddie Stark: 'Morning! Joy Stark...,62-0-6,Because we owe them 1200.,"[""[first lines] Eddie Stark: 'Morning! Joy Sta...",[Because we owe them 1200.]
2,FBI Agent Nicole Scott: You want another crool...,75-1-3,I've had two already. Who eats three croolers ...,['FBI Agent Nicole Scott: You want another cro...,[I've had two already. Who eats three croolers...
3,Janey Cooper: Would you like a muffin? Jess Ma...,75-5-4,Try one.,['Janey Cooper: Would you like a muffin? Jess ...,[What is the best way to try one?]
4,Lucky: Red Airedale rules! Cadpig: Yellow's my...,107-5-2,I like the brown one. It looks like gravy.,"[""Lucky: Red Airedale rules! Cadpig: Yellow's ...",[I like the brown one. It looks like gravy.]
...,...,...,...,...,...
295,Sheriff Neil Prescott: It's against the law to...,495-1-4,A man in trouble generally runs to his friends...,['Sheriff Neil Prescott: What makes you think ...,[A man in trouble generally runs to his friend...
296,Ranger Clint Travis: The sign outside says you...,500-1-3,You can find an easier one - being a ranger is...,['Ranger Clint Travis: The sign outside says y...,[You can find an easier one - being a ranger i...
297,Banker: The Papago chiefs distributed the mone...,505-0-1,Or a dry one.,['Banker: The Papago Chiefs distributed the mo...,[Or a dry one.]
298,"President George W Bush: General, call off the...",508-8-10,Yeah the speedy one!,"['President George W Bush: General, call off t...",[Yeah the speedy one!]


In [ ]:
data2.to_csv('/content/gdrive/My Drive/Colab Notebooks/Bart_for_sentences_and_context.tsv')

In [ ]:
data2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Bart_for_sentences_and_context.tsv', index_col=False, error_bad_lines = False)
data2.head(5)

,Unnamed: 0,Paragraph,Id,Sentence,BART_Paragraph,BART_Sentence
0,0,"[first lines] Joy Stark: Okay, remember that t...",58-1-1,"Okay, remember that the garbage goes out on Tu...","['Joy Stark: Okay, remember that the garbage g...","['Okay, remember that the garbage goes out on ..."
1,1,[first lines] Eddie Stark: 'Morning! Joy Stark...,62-0-6,Because we owe them 1200.,"[""[first lines] Eddie Stark: 'Morning! Joy Sta...",['Because we owe them 1200.']
2,2,FBI Agent Nicole Scott: You want another crool...,75-1-3,I've had two already. Who eats three croolers ...,['FBI Agent Nicole Scott: You want another cro...,"[""I've had two already. Who eats three crooler..."
3,3,Janey Cooper: Would you like a muffin? Jess Ma...,75-5-4,Try one.,['Janey Cooper: Would you like a muffin? Jess ...,['What is the best way to try one?']
4,4,Lucky: Red Airedale rules! Cadpig: Yellow's my...,107-5-2,I like the brown one. It looks like gravy.,"[""Lucky: Red Airedale rules! Cadpig: Yellow's ...",['I like the brown one. It looks like gravy.']


In [ ]:
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)

In [ ]:
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

In [ ]:
Parrot_list =[]
for i in data2['Paragraph']:
  Parrot_list.append(parrot.augment(input_phrase=i))

Token indices sequence length is longer than the specified maximum sequence length for this model (1537 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
data2['Perrot'] = Parrot_list

In [ ]:
data2= data2.rename(columns={'Perrot': 'Parrot'})

In [ ]:
data2

,Unnamed: 0,Paragraph,Id,Sentence,BART_Paragraph,BART_Sentence,Parrot
0,0,"[first lines] Joy Stark: Okay, remember that t...",58-1-1,"Okay, remember that the garbage goes out on Tu...","['Joy Stark: Okay, remember that the garbage g...","['Okay, remember that the garbage goes out on ...","[([first lines] Joy Stark: Okay, remember that..."
1,1,[first lines] Eddie Stark: 'Morning! Joy Stark...,62-0-6,Because we owe them 1200.,"[""[first lines] Eddie Stark: 'Morning! Joy Sta...",['Because we owe them 1200.'],None
2,2,FBI Agent Nicole Scott: You want another crool...,75-1-3,I've had two already. Who eats three croolers ...,['FBI Agent Nicole Scott: You want another cro...,"[""I've had two already. Who eats three crooler...",None
3,3,Janey Cooper: Would you like a muffin? Jess Ma...,75-5-4,Try one.,['Janey Cooper: Would you like a muffin? Jess ...,['What is the best way to try one?'],None
4,4,Lucky: Red Airedale rules! Cadpig: Yellow's my...,107-5-2,I like the brown one. It looks like gravy.,"[""Lucky: Red Airedale rules! Cadpig: Yellow's ...",['I like the brown one. It looks like gravy.'],None
...,...,...,...,...,...,...,...
295,295,Sheriff Neil Prescott: It's against the law to...,495-1-4,A man in trouble generally runs to his friends...,['Sheriff Neil Prescott: What makes you think ...,['A man in trouble generally runs to his frien...,[(Sheriff Neil Prescott: It's against the law ...
296,296,Ranger Clint Travis: The sign outside says you...,500-1-3,You can find an easier one - being a ranger is...,['Ranger Clint Travis: The sign outside says y...,"[""You can find an easier one - being a ranger ...",None
297,297,Banker: The Papago chiefs distributed the mone...,505-0-1,Or a dry one.,['Banker: The Papago Chiefs distributed the mo...,['Or a dry one.'],[(Banker: The Papago chiefs distributed the mo...
298,298,"President George W Bush: General, call off the...",508-8-10,Yeah the speedy one!,"['President George W Bush: General, call off t...",['Yeah the speedy one!'],None


In [ ]:
data2.to_csv('/content/gdrive/My Drive/Colab Notebooks/BART_and_PARROT_context_3.tsv')

In [ ]:
data3 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/BART_and_PARROT_context_3.tsv', index_col=False, error_bad_lines = False)
data3.head(5)

,Unnamed: 0,Unnamed: 0.1,Paragraph,Id,Sentence,BART_Paragraph,BART_Sentence,Parrot
0,0,0,"[first lines] Joy Stark: Okay, remember that t...",58-1-1,"Okay, remember that the garbage goes out on Tu...","['Joy Stark: Okay, remember that the garbage g...","['Okay, remember that the garbage goes out on ...","[(""[first lines] Joy Stark: Okay, remember tha..."
1,1,1,[first lines] Eddie Stark: 'Morning! Joy Stark...,62-0-6,Because we owe them 1200.,"[""[first lines] Eddie Stark: 'Morning! Joy Sta...",['Because we owe them 1200.'],NaN
2,2,2,FBI Agent Nicole Scott: You want another crool...,75-1-3,I've had two already. Who eats three croolers ...,['FBI Agent Nicole Scott: You want another cro...,"[""I've had two already. Who eats three crooler...",NaN
3,3,3,Janey Cooper: Would you like a muffin? Jess Ma...,75-5-4,Try one.,['Janey Cooper: Would you like a muffin? Jess ...,['What is the best way to try one?'],NaN
4,4,4,Lucky: Red Airedale rules! Cadpig: Yellow's my...,107-5-2,I like the brown one. It looks like gravy.,"[""Lucky: Red Airedale rules! Cadpig: Yellow's ...",['I like the brown one. It looks like gravy.'],NaN


In [ ]:
# Pegasus mdoel:
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
pegasus_list =[]
for i in data3['Paragraph']:
  pegasus_list.append(get_response(i, 1))

data3['Pegasus'] = pegasus_list

In [ ]:
data3

,Unnamed: 0,Unnamed: 0.1,Paragraph,Id,Sentence,BART_Paragraph,BART_Sentence,Parrot,Pegasus
0,0,0,"[first lines] Joy Stark: Okay, remember that t...",58-1-1,"Okay, remember that the garbage goes out on Tu...","['Joy Stark: Okay, remember that the garbage g...","['Okay, remember that the garbage goes out on ...","[(""[first lines] Joy Stark: Okay, remember tha...","[If you see a man in the bushes, please don't ..."
1,1,1,[first lines] Eddie Stark: 'Morning! Joy Stark...,62-0-6,Because we owe them 1200.,"[""[first lines] Eddie Stark: 'Morning! Joy Sta...",['Because we owe them 1200.'],NaN,[Eddie Stark asked why Joy Stark was paying th...
2,2,2,FBI Agent Nicole Scott: You want another crool...,75-1-3,I've had two already. Who eats three croolers ...,['FBI Agent Nicole Scott: You want another cro...,"[""I've had two already. Who eats three crooler...",NaN,"[FBI Agent Nicole Scott: Thank you very much, ..."
3,3,3,Janey Cooper: Would you like a muffin? Jess Ma...,75-5-4,Try one.,['Janey Cooper: Would you like a muffin? Jess ...,['What is the best way to try one?'],NaN,[Janey Cooper wants a muffin. Jess Mastriani d...
4,4,4,Lucky: Red Airedale rules! Cadpig: Yellow's my...,107-5-2,I like the brown one. It looks like gravy.,"[""Lucky: Red Airedale rules! Cadpig: Yellow's ...",['I like the brown one. It looks like gravy.'],NaN,[Cadpig likes the yellow one. Rolly likes the ...
...,...,...,...,...,...,...,...,...,...
295,295,295,Sheriff Neil Prescott: It's against the law to...,495-1-4,A man in trouble generally runs to his friends...,['Sheriff Neil Prescott: What makes you think ...,['A man in trouble generally runs to his frien...,"[(""Sheriff Neil Prescott: It's against the law...",[It's against the law to help a wanted man.]
296,296,296,Ranger Clint Travis: The sign outside says you...,500-1-3,You can find an easier one - being a ranger is...,['Ranger Clint Travis: The sign outside says y...,"[""You can find an easier one - being a ranger ...",NaN,[Ranger Clint Travis: I need a job.]
297,297,297,Banker: The Papago chiefs distributed the mone...,505-0-1,Or a dry one.,['Banker: The Papago Chiefs distributed the mo...,['Or a dry one.'],[('Banker: The Papago chiefs distributed the m...,[Most of the certificates found their way back...
298,298,298,"President George W Bush: General, call off the...",508-8-10,Yeah the speedy one!,"['President George W Bush: General, call off t...",['Yeah the speedy one!'],NaN,"[President George W Bush: General, stop lookin..."


In [ ]:
data3.to_csv('/content/gdrive/My Drive/Colab Notebooks/BART_and_PARROT_Pegasus_context_3.tsv')

# Scores for BART model

In [ ]:
rouge = ROUGEScore()

In [ ]:
print(len(data2['Paragraph']))
print(len(data2['BART_Paragraph']))

300
300


In [ ]:
def get_bleu_score(x, y):
  return sentence_bleu(x,y)
data2['Bleu_BART_Sentence'] =  data2.apply(lambda x: get_bleu_score(x['Sentence'], x['BART_Sentence']) if x['BART_Sentence'] != None  else print("Not available"), axis=1)


In [ ]:
def get_bleu_score_corpus(x, y):
  return corpus_bleu(x,y)

data2['Bleu_BART_Paragraph'] =  data2.apply(lambda x: get_bleu_score_corpus([x['Paragraph']], [x['BART_Paragraph']]) if x['BART_Paragraph'] != None  else print("Not available"), axis=1)

In [ ]:
sentence_bleu(data2['Sentence'][0], data2['BART_Sentence'][0])

1.3072073186324436e-231

In [ ]:
def get_rouge_score(x, y):
  return rouge(x,y)

data2['Rouge_BART_Paragraph'] =  data2.apply(lambda x: get_rouge_score(x['Paragraph'], x['BART_Paragraph']) if x['BART_Paragraph'] != None  else print("Not available"), axis=1)
data2['Rouge_BART_Sentence'] =  data2.apply(lambda x: get_rouge_score(x['Sentence'], x['BART_Sentence']) if x['BART_Sentence'] != None  else print("Not available"), axis=1)

In [ ]:
data2

,Unnamed: 0,Paragraph,Id,Sentence,BART_Paragraph,BART_Sentence,Bleu_BART_Sentence,Bleu_BART_Paragraph,Rouge_BART_Paragraph,Rouge_BART_Sentence
0,0,"[first lines] Joy Stark: Okay, remember that t...",58-1-1,"Okay, remember that the garbage goes out on Tu...","['Joy Stark: Okay, remember that the garbage g...","['Okay, remember that the garbage goes out on ...",1.307207e-231,1.375877e-231,"{'rouge1_fmeasure': tensor(0.3232), 'rouge1_pr...","{'rouge1_fmeasure': tensor(0.4595), 'rouge1_pr..."
1,1,[first lines] Eddie Stark: 'Morning! Joy Stark...,62-0-6,Because we owe them 1200.,"[""[first lines] Eddie Stark: 'Morning! Joy Sta...",['Because we owe them 1200.'],1.570142e-231,1.438682e-231,"{'rouge1_fmeasure': tensor(0.4074), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."
2,2,FBI Agent Nicole Scott: You want another crool...,75-1-3,I've had two already. Who eats three croolers ...,['FBI Agent Nicole Scott: You want another cro...,"[""I've had two already. Who eats three crooler...",1.411827e-231,1.458673e-231,"{'rouge1_fmeasure': tensor(0.1642), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."
3,3,Janey Cooper: Would you like a muffin? Jess Ma...,75-5-4,Try one.,['Janey Cooper: Would you like a muffin? Jess ...,['What is the best way to try one?'],1.164047e-231,1.479965e-231,"{'rouge1_fmeasure': tensor(0.2000), 'rouge1_pr...","{'rouge1_fmeasure': tensor(0.4000), 'rouge1_pr..."
4,4,Lucky: Red Airedale rules! Cadpig: Yellow's my...,107-5-2,I like the brown one. It looks like gravy.,"[""Lucky: Red Airedale rules! Cadpig: Yellow's ...",['I like the brown one. It looks like gravy.'],1.460519e-231,1.519043e-231,"{'rouge1_fmeasure': tensor(0.6429), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."
...,...,...,...,...,...,...,...,...,...,...
295,295,Sheriff Neil Prescott: It's against the law to...,495-1-4,A man in trouble generally runs to his friends...,['Sheriff Neil Prescott: What makes you think ...,['A man in trouble generally runs to his frien...,1.358882e-231,1.384293e-231,"{'rouge1_fmeasure': tensor(0.2439), 'rouge1_pr...","{'rouge1_fmeasure': tensor(0.5556), 'rouge1_pr..."
296,296,Ranger Clint Travis: The sign outside says you...,500-1-3,You can find an easier one - being a ranger is...,['Ranger Clint Travis: The sign outside says y...,"[""You can find an easier one - being a ranger ...",1.320869e-231,1.366427e-231,"{'rouge1_fmeasure': tensor(0.1783), 'rouge1_pr...","{'rouge1_fmeasure': tensor(0.4250), 'rouge1_pr..."
297,297,Banker: The Papago chiefs distributed the mone...,505-0-1,Or a dry one.,['Banker: The Papago Chiefs distributed the mo...,['Or a dry one.'],1.595497e-231,1.326727e-231,"{'rouge1_fmeasure': tensor(0.4828), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."
298,298,"President George W Bush: General, call off the...",508-8-10,Yeah the speedy one!,"['President George W Bush: General, call off t...",['Yeah the speedy one!'],1.562937e-231,1.396936e-231,"{'rouge1_fmeasure': tensor(0.1296), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."


In [ ]:
data2.to_csv('/content/gdrive/My Drive/Colab Notebooks/Scores_for_Bart_for_sentences_and_context_2.tsv')

# Taking a random sample

In [ ]:
data2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Scores_for_Bart_for_sentences_and_context_2.tsv', index_col=False, error_bad_lines = False)
data2.head(5)

,Unnamed: 0,Unnamed: 0.1,Paragraph,Id,Sentence,BART_Paragraph,BART_Sentence,Bleu_BART_Sentence,Bleu_BART_Paragraph,Rouge_BART_Paragraph,Rouge_BART_Sentence
0,0,0,"[first lines] Joy Stark: Okay, remember that t...",58-1-1,"Okay, remember that the garbage goes out on Tu...","['Joy Stark: Okay, remember that the garbage g...","['Okay, remember that the garbage goes out on ...",1.307207e-231,1.375877e-231,"{'rouge1_fmeasure': tensor(0.3232), 'rouge1_pr...","{'rouge1_fmeasure': tensor(0.4595), 'rouge1_pr..."
1,1,1,[first lines] Eddie Stark: 'Morning! Joy Stark...,62-0-6,Because we owe them 1200.,"[""[first lines] Eddie Stark: 'Morning! Joy Sta...",['Because we owe them 1200.'],1.570142e-231,1.438682e-231,"{'rouge1_fmeasure': tensor(0.4074), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."
2,2,2,FBI Agent Nicole Scott: You want another crool...,75-1-3,I've had two already. Who eats three croolers ...,['FBI Agent Nicole Scott: You want another cro...,"[""I've had two already. Who eats three crooler...",1.411827e-231,1.458673e-231,"{'rouge1_fmeasure': tensor(0.1642), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."
3,3,3,Janey Cooper: Would you like a muffin? Jess Ma...,75-5-4,Try one.,['Janey Cooper: Would you like a muffin? Jess ...,['What is the best way to try one?'],1.164047e-231,1.479965e-231,"{'rouge1_fmeasure': tensor(0.2000), 'rouge1_pr...","{'rouge1_fmeasure': tensor(0.4000), 'rouge1_pr..."
4,4,4,Lucky: Red Airedale rules! Cadpig: Yellow's my...,107-5-2,I like the brown one. It looks like gravy.,"[""Lucky: Red Airedale rules! Cadpig: Yellow's ...",['I like the brown one. It looks like gravy.'],1.460519e-231,1.519043e-231,"{'rouge1_fmeasure': tensor(0.6429), 'rouge1_pr...","{'rouge1_fmeasure': tensor(1.), 'rouge1_precis..."


In [ ]:
data_sampled = data2.sample(n = 50)

In [ ]:
data_sampled.to_csv('/content/gdrive/My Drive/Colab Notebooks/sample_sentences_and_context_3.tsv')